In [5]:
%load_ext autoreload
%autoreload

import serial
import time

import struct
import numpy as np

# Change it to COMx on a MS Windows platform
serUSB = '/dev/tty.usbmodem1454301'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Test serial communication

In [6]:
with serial.Serial(serUSB, 9600, timeout=1) as ser:
    ser.write(b'101,2')

In [10]:
import drv_gpio as drv

ser = serial.Serial('/dev/tty.usbmodem1454301', 9600, timeout=1)
drv.connect(ser)

Write port H with value 0b0101

In [12]:
drv.gpio_port_write('H', 0b0101)